In [ ]:
%load_ext autoreload
%autoreload 1
%aimport ParallelEnvironment
%aimport Worker
%aimport Estimator_Pong

In [ ]:
import numpy as np
import ParallelEnvironment
import Worker
import Estimator_Cliff_Walk
from ParallelEnvironment import *
from Worker import *
from Estimator_Pong import *

In [ ]:
import tensorflow as tf
import numpy as np


In [ ]:
import gym

In [5]:
tf.reset_default_graph()
provider = EnvironmentProvider("Pong-v0",True,[2,3],True,1,-1)
parallel = ParallelEnvironment(provider, 32, 8, 4)

[2017-06-09 17:45:44,370] Making new env: Pong-v0
[2017-06-09 17:45:44,509] Making new env: Pong-v0
[2017-06-09 17:45:44,641] Making new env: Pong-v0
[2017-06-09 17:45:44,773] Making new env: Pong-v0
[2017-06-09 17:45:44,907] Making new env: Pong-v0
[2017-06-09 17:45:45,039] Making new env: Pong-v0
[2017-06-09 17:45:45,172] Making new env: Pong-v0
[2017-06-09 17:45:45,305] Making new env: Pong-v0
[2017-06-09 17:45:45,436] Making new env: Pong-v0
[2017-06-09 17:45:45,569] Making new env: Pong-v0
[2017-06-09 17:45:45,701] Making new env: Pong-v0
[2017-06-09 17:45:45,832] Making new env: Pong-v0
[2017-06-09 17:45:45,965] Making new env: Pong-v0
[2017-06-09 17:45:46,097] Making new env: Pong-v0
[2017-06-09 17:45:46,228] Making new env: Pong-v0
[2017-06-09 17:45:46,362] Making new env: Pong-v0
[2017-06-09 17:45:46,494] Making new env: Pong-v0
[2017-06-09 17:45:46,626] Making new env: Pong-v0
[2017-06-09 17:45:46,757] Making new env: Pong-v0
[2017-06-09 17:45:46,890] Making new env: Pong-v0


In [ ]:
estimator = Estimator(2,0.001)

In [7]:
def get_epsilon(step):
    return 0.01

In [8]:
def calculate_q_targets(estimator, states, rewards, done):
    next_max_q_values = np.max(estimator.predict_q_values(states), axis= 1)
    next_max_q_values[done == 1] = 0
    q_targets = rewards + next_max_q_values
    return q_targets

In [9]:
def train(env, estimator, num_global_steps):
    shared_states, shared_rewards, shared_done, shared_actions = env.get_shared_variables()
    
    for i in range(num_global_steps):
        if ((i+1) % 1000) == 0:
            print("curr step: {}".format(i+1))
            print("loss: {}".format(loss))
        epsilon = get_epsilon(i)
        actions = estimator.choose_e_greedy_actions(shared_states, epsilon)
        old_states = np.copy(shared_states)
        #print("shared states before actions:")
        #print(shared_states)
        #print("------------ \n")
        #print("actions to be performed: ")
        #print(actions)
        #print("------------ \n")
        
        # rescale rewards
        
        
        
        
        env.step(actions)
        #print("check whether actions and internal actions are equivalent: ")
        #print(np.argmax(shared_actions) == actions)
        #print("external actions: ")
        #print(actions)
        #print("internal actions")
        #print(shared_actions)
        #print("------------ \n")
        #print("shared states:")
        #print(shared_states)
        #print("------------ \n")
        q_targets = calculate_q_targets(estimator, shared_states, shared_rewards, shared_done)
        #print("q targets: ")
        #print(q_targets)
        loss = estimator.update(old_states, actions, q_targets)
        #print("loss: "+ str(loss))
    return estimator
        
        
        

In [16]:
tf.global_variables()

[<tf.Variable 'conv_1/kernel:0' shape=(8, 8, 1, 16) dtype=float32_ref>,
 <tf.Variable 'conv_1/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'conv_2/kernel:0' shape=(4, 4, 16, 32) dtype=float32_ref>,
 <tf.Variable 'conv_2/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(2592, 256) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'out/kernel:0' shape=(256, 2) dtype=float32_ref>,
 <tf.Variable 'out/bias:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'conv_1/kernel/Adam:0' shape=(8, 8, 1, 16) dtype=float32_ref>,
 <tf.Variable 'conv_1/kernel/Adam_1:0' shape=(8, 8, 1, 16) dtype=float32_ref>,
 <tf.Variable 'conv_1/bias/Adam:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'conv_1/bias/Adam_1:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'conv_2/kernel/Adam:0' shape=(4, 4, 16, 32) dtype=

In [11]:
estimator = train(parallel, estimator, 100)

In [13]:
import time

In [18]:
def play(estimator, env):
    state = env.reset()
    done = False
    while not done:
        action = estimator.choose_greedy_actions([state])[0]
        state, reward, done = env.step(action)
        env.render()
        time.sleep(0.1)

In [19]:
envi = provider.create()

[2017-06-09 17:36:40,939] Making new env: Pong-v0


In [20]:
play(estimator, envi)

KeyboardInterrupt: 

In [49]:
%%timeit
if a > 1:
    b = 1

10000000 loops, best of 3: 31.3 ns per loop
